# Homework: Data talks club data engineering zoomcamp Data loading workshop
Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do

## 1. Use a generator
Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

Answer the following questions:

Question 1: What is the sum of the outputs of the generator for limit = 5?
Question 2: What is the 13th number yielded
I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

# Calculate the sum of the outputs
sum_of_outputs = sum(generator)

print("Sum of the square roots:", sum_of_outputs)

Sum of the square roots: 8.382332347441762


In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Find the 13th number yielded
limit = 13
generator = square_root_generator(limit)

# Iterate through the generator until the 13th value
for i, value in enumerate(generator, 1):
    if i == 13:
        print("The 13th number yielded is:", value)
        break

The 13th number yielded is: 3.605551275463989


## 2. Append a generator to a table with existing data
Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. After correctly appending the data, calculate the sum of all ages of people.

In [3]:
%pip install -U dlt

  Using cached dlt-0.4.2-py3-none-any.whl.metadata (9.5 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached GitPython-3.1.41-py3-none-any.whl.metadata (14 kB)
  Using cached giturlparse-0.12.0-py2.py3-none-any.whl.metadata (4.5 kB)
  Using cached hexbytes-1.0.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached humanize-4.9.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonpath_ng-1.6.1-py3-none-any.whl.metadata (18 kB)
  Using cached makefun-1.15.2-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached orjson-3.9.13-cp311-cp311-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata (49 kB)
  Using cached pathvalidate-3.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached requirements_parser-0.5.0-py3-none-any.whl (18 kB)
  Using cached semver-3.0.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached setuptools-69.0.3-py3-none-any.whl.metadata (6.3 kB)
  Using cached toml

In [3]:
%pip install -U duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 40.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')
load_info = pipeline.run(people_1(),
                        table_name="persons",
                        write_disposition="replace")

print(load_info)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
Pipeline dlt_ipykernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/nevenkalukic/code/dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707217875.303942 is LOADED and contains no failed jobs


In [5]:
# show the outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n Rides table below: Note the times are properly typed")
persons = conn.sql("SELECT * FROM persons").df()
display(persons)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons             │
└─────────────────────┘




 Rides table below: Note the times are properly typed


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707217875.303942,d29Iyq/0RrEQRA
1,2,Person_2,27,City_A,1707217875.303942,YvFcEev5e0sZUQ
2,3,Person_3,28,City_A,1707217875.303942,VQqANeWK8ooQ3Q
3,4,Person_4,29,City_A,1707217875.303942,5arDKvXQerrqIQ
4,5,Person_5,30,City_A,1707217875.303942,4iCXET7qt7KmNw


In [6]:


# Initialize sum of ages
sum_of_ages = 0

for person in people_1():
    sum_of_ages += person["Age"]

print("Sum of ages of all people:", sum_of_ages)


Sum of ages of all people: 140


In [7]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

load_info = pipeline.run(people_2(),
                        table_name="persons",
                        write_disposition="append")

print(load_info)

{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Pipeline dlt_ipykernel_launcher load step completed in 0.15 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/nevenkalukic/code/dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707218127.5204759 is LOADED and contains no failed jobs


In [8]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n Persons table below: Note the times are properly typed")
persons = conn.sql("SELECT * FROM persons").df()
display(persons)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons             │
└─────────────────────┘




 Rides table below: Note the times are properly typed


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707217875.303942,d29Iyq/0RrEQRA,None
1,2,Person_2,27,City_A,1707217875.303942,YvFcEev5e0sZUQ,None
2,3,Person_3,28,City_A,1707217875.303942,VQqANeWK8ooQ3Q,None
3,4,Person_4,29,City_A,1707217875.303942,5arDKvXQerrqIQ,None
4,5,Person_5,30,City_A,1707217875.303942,4iCXET7qt7KmNw,None
5,3,Person_3,33,City_B,1707218127.5204759,hzbvFPlG87uXdw,Job_3
6,4,Person_4,34,City_B,1707218127.5204759,DMfZMoUS956TEg,Job_4
7,5,Person_5,35,City_B,1707218127.5204759,3m821h6Io5WpfQ,Job_5
8,6,Person_6,36,City_B,1707218127.5204759,YMyIDlT188KSdA,Job_6
9,7,Person_7,37,City_B,1707218127.5204759,Hxr62UhvUZs7vA,Job_7


In [9]:
joined = conn.sql("""
SELECT SUM("age") as all_ages
FROM persons 
""").df()
display(joined)

,all_ages
0,353.0


## 3. Merge a generator
Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

 Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: Calculate the sum of ages of all the people loaded as described above.

In [10]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

pipeline = dlt.pipeline(destination='duckdb', dataset_name='citizens')
load_info = pipeline.run(people_1(),
                        table_name="members",
                        write_disposition="replace",
                        primary_key='id')

print(load_info)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
Pipeline dlt_ipykernel_launcher load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset citizens
The duckdb destination used duckdb:////Users/nevenkalukic/code/dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707219454.249002 is LOADED and contains no failed jobs


In [11]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n Members table below: Note the times are properly typed")
members = conn.sql("SELECT * FROM members").df()
display(members)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ members             │
└─────────────────────┘




 Members table below: Note the times are properly typed


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707219454.249002,istLq0Od19mTow
1,2,Person_2,27,City_A,1707219454.249002,fRuSz43p7Z9H8A
2,3,Person_3,28,City_A,1707219454.249002,IxytKEu9jbjzSA
3,4,Person_4,29,City_A,1707219454.249002,CcFKZYEyTvGyHQ
4,5,Person_5,30,City_A,1707219454.249002,ZJ8eegNjX3m/qQ


In [12]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

load_info = pipeline.run(people_2(),
                        table_name="members",
                        write_disposition="merge",
                        primary_key='id')

print(load_info)


{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Pipeline dlt_ipykernel_launcher load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset citizens
The duckdb destination used duckdb:////Users/nevenkalukic/code/dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707219482.1309361 is LOADED and contains no failed jobs


In [13]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n Members table below: Note the times are properly typed")
members = conn.sql("SELECT * FROM members").df()
display(members)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ members             │
└─────────────────────┘




 Members table below: Note the times are properly typed


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707219454.249002,istLq0Od19mTow,None
1,2,Person_2,27,City_A,1707219454.249002,fRuSz43p7Z9H8A,None
2,7,Person_7,37,City_B,1707219482.1309361,f554w4iIY9of5g,Job_7
3,3,Person_3,33,City_B,1707219482.1309361,kASpfnAE5/t/HA,Job_3
4,6,Person_6,36,City_B,1707219482.1309361,5Od40f9KXrJNmA,Job_6
5,8,Person_8,38,City_B,1707219482.1309361,28u1I5v7rsT0VA,Job_8
6,5,Person_5,35,City_B,1707219482.1309361,jVlTl7Sx4E39Hw,Job_5
7,4,Person_4,34,City_B,1707219482.1309361,pXSMk00hSrAn0g,Job_4


In [14]:
joined = conn.sql("""
SELECT SUM("age") as all_ages
FROM members 
""").df()
display(joined)

,all_ages
0,266.0
